In [25]:
import numpy as np
import plotly.graph_objects as go

# Paramètres de simulation
Te = 1.0               # pas de temps
T = 100               # nombre d'instants
sigma_theta = 0.03    # écart type sur l'angle (en radians)
sigma_r = 20           # écart type sur la distance (en mètres)
sigma_Q = 1.0         # niveau de bruit de processus

# État initial : [px, vx, py, vy]
x_init = np.array([3, 40, -4, 20])
x_true = np.zeros((4, T))
x_true[:, 0] = x_init

# Matrice de transition F (modèle vitesse constante)
F = np.array([
    [1, Te, 0,  0],
    [0, 1,  0,  0],
    [0, 0,  1, Te],
    [0, 0,  0, 1]
])

# Bruit de processus (matrice Q)
Q = sigma_Q**2 * np.array([
    [Te**3/3, Te**2/2, 0, 0],
    [Te**2/2, Te,      0, 0],
    [0, 0, Te**3/3, Te**2/2],
    [0, 0, Te**2/2, Te]
])

# Bruit de mesure en coordonnées polaires (matrice R)
R = np.diag([sigma_theta**2, sigma_r**2])

# Fonction d'observation non linéaire : retourne (angle, distance)
def h(x):
    px, _, py, _ = x
    theta = np.arctan2(py, px)
    r = np.sqrt(px**2 + py**2)
    return np.array([theta, r])

# Jacobien de la fonction h(x)
def jacobian_h(x):
    px, _, py, _ = x
    rho2 = px**2 + py**2
    rho = np.sqrt(rho2)
    if rho2 == 0:
        return np.zeros((2, 4))  # cas limite à gérer
    H = np.array([
        [-py / rho2, 0, px / rho2, 0],
        [ px / rho,  0, py / rho,  0]
    ])
    return H

# Simulation de la trajectoire réelle
for k in range(1, T):
    bruit = np.random.multivariate_normal(np.zeros(4), Q)
    x_true[:, k] = F @ x_true[:, k-1] + bruit

# Génération des observations bruitées en (theta, r)
y = np.zeros((2, T))
for k in range(T):
    bruit = np.random.multivariate_normal(np.zeros(2), R)
    y[:, k] = h(x_true[:, k]) + bruit

# Boucle du filtre de Kalman étendu (EKF)
x_est = np.zeros((4, T))
x_est[:, 0] = x_init
P = np.eye(4)  # incertitude initiale

for k in range(1, T):
    # Prédiction
    x_pred = F @ x_est[:, k-1]
    P_pred = F @ P @ F.T + Q

    # Mise à jour
    Hk = jacobian_h(x_pred)
    y_pred = h(x_pred)
    S = Hk @ P_pred @ Hk.T + R
    K = P_pred @ Hk.T @ np.linalg.inv(S)
    innovation = y[:, k] - y_pred
    x_est[:, k] = x_pred + K @ innovation
    P = (np.eye(4) - K @ Hk) @ P_pred

# Affichage avec Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true[0], y=x_true[2], mode='lines', name='Trajectoire réelle'))

# Conversion des observations polaires en cartésiennes pour affichage
obs_px = y[1] * np.cos(y[0])
obs_py = y[1] * np.sin(y[0])
fig.add_trace(go.Scatter(x=obs_px, y=obs_py, mode='markers', name='Observations'))

fig.add_trace(go.Scatter(x=x_est[0], y=x_est[2], mode='lines', name='Estimation EKF'))

fig.update_layout(title="Suivi de la trajectoire avec EKF sur des données polaires",
                  xaxis_title="Position x", yaxis_title="Position y",
                  width=800, height=500)
fig.show()


In [26]:
fig_theta = go.Figure()
fig_theta.add_trace(go.Scatter(
    y=[h(x_true[:, k])[0] for k in range(T)],
    mode='lines',
    name='θ réel',
    line=dict(color='blue')  # bleu
))
fig_theta.add_trace(go.Scatter(
    y=y[0],
    mode='markers',
    name='θ observé',
    line=dict(color='red')  # orange
))
fig_theta.add_trace(go.Scatter(
    y=[h(x_est[:, k])[0] for k in range(T)],
    mode='lines',
    name='θ estimé',
    line=dict(color='green')  # vert
))
fig_theta.update_layout(
    title="Évolution de l'angle θ (en radians)",
    xaxis_title="Temps (k)",
    yaxis_title="θ (rad)",
    
)
fig_theta.show()


In [28]:
fig_r = go.Figure()
fig_r.add_trace(go.Scatter(y=[h(x_true[:, k])[1] for k in range(T)],
                           mode='markers',
                           name='r réel',
                           line=dict(color='blue')))
fig_r.add_trace(go.Scatter(y=y[1],
                           mode='markers',
                           name='r observé',
                           line=dict(color='red')))
fig_r.add_trace(go.Scatter(y=[h(x_est[:, k])[1] for k in range(T)],
                           mode='lines',
                           name='r estimé',
                           line=dict(color='green')))
fig_r.update_layout(title="Évolution de la distance r",
                    xaxis_title="Temps (k)",
                    yaxis_title="r (mètres)",
                    width=800, height=500)
fig_r.show()
